# Activity 3: Binary Classification on MNIST

**Focus:** Compare different binary classifiers and measure their performance.

**Task:** Identify digits that are **NOT 3 or 5** (binary classification).

## Setup

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score,
    f1_score, roc_auc_score, roc_curve, accuracy_score
)


## Load MNIST Dataset

In [ ]:

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target'].astype(int)

# Binary target: 1 if NOT (3 or 5), else 0
y_binary = np.where((y != 3) & (y != 5), 1, 0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)


## SGD Classifier (Baseline)

In [ ]:

sgd_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sgd', SGDClassifier(random_state=42))
])

sgd_pipeline.fit(X_train, y_train)
y_pred_sgd = sgd_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_sgd))
print("Precision:", precision_score(y_test, y_pred_sgd))
print("Recall:", recall_score(y_test, y_pred_sgd))
print("F1:", f1_score(y_test, y_pred_sgd))


## Fine-tuning SGD using Grid Search

In [ ]:

param_grid = {
    'sgd__loss': ['hinge', 'log_loss'],
    'sgd__alpha': [1e-4, 1e-3],
    'sgd__max_iter': [1000, 2000]
}

grid_search = GridSearchCV(
    sgd_pipeline, param_grid, scoring='f1', cv=3, n_jobs=-1
)
grid_search.fit(X_train, y_train)

best_sgd = grid_search.best_estimator_
y_pred_best_sgd = best_sgd.predict(X_test)

print("Best Params:", grid_search.best_params_)
print("F1 (Tuned SGD):", f1_score(y_test, y_pred_best_sgd))


## Logistic Regression

In [ ]:

logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

logreg_pipeline.fit(X_train, y_train)
y_pred_logreg = logreg_pipeline.predict(X_test)

print("F1 (Logistic Regression):", f1_score(y_test, y_pred_logreg))


## Support Vector Machine (SVM)

In [ ]:

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', probability=True))
])

svm_pipeline.fit(X_train, y_train)
y_pred_svm = svm_pipeline.predict(X_test)

print("F1 (SVM):", f1_score(y_test, y_pred_svm))


## Random Forest

In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("F1 (Random Forest):", f1_score(y_test, y_pred_rf))


## Performance Comparison Summary


- **SGD (baseline):** Fast and scalable, but sensitive to hyperparameters.
- **Tuned SGD:** Improved F1 score after hyperparameter tuning.
- **Logistic Regression:** Stable and interpretable, strong baseline.
- **SVM:** High performance but computationally expensive.
- **Random Forest:** Handles non-linearity well but slower and less interpretable.

**Key Metrics Used:**
- Precision & Recall: Important due to class imbalance.
- F1-score: Balances precision and recall.
- Accuracy: Less informative alone.
- ROC-AUC: Measures ranking performance.
